# MongoDB with python

In [1]:
%matplotlib inline
import pymongo
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure
from bson import json_util, ObjectId
import pandas as pd
from pandas import DataFrame
from pandas.io.json import json_normalize
import numpy as np
import requests
import json, os
import configparser
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import seaborn as sns
import warnings
import random
import pprint
from datetime import datetime
random.seed(datetime.now())
warnings.filterwarnings('ignore')

# Make plots larger
plt.rcParams['figure.figsize'] = (10, 6)

###  Install PyMongo¶

Linux/OS X   
You can use pip to install:  

```bash  
pip install pymongo
```

###  Create a Connection  


Use MongoClient to create a connection:

```python  
client = MongoClient()
```
If you do not specify any arguments to MongoClient, then MongoClient defaults to the MongoDB instance that runs on the localhost interface on port 27017.   

You can also specify a complete MongoDB URI to define the connection, including explicitly specifying the host and port number. For example, the following creates a connection to a MongoDB instance that runs on mongodb0.example.net and the port of 27017:  


```python
client = MongoClient("mongodb://mongodb0.example.net:27017")
```

### Access Database Objects  

The first fundamental class of objects you will interact with using pymongo is Database which represents the database construct in MongoDB. Databases hold groups of logically related collections. MongoDB creates new databases implicitly upon their first use.   

To assign the database named primer to the local variable db, you can use attribute access, as in the following:  

```python
db = client.primer
```
You can also access databases using dictionary-style access, which removes Python-specific naming restrictions, as in the following:  

```python
db = client['primer']
```

### Access Collection Objects   

The second fundamental class of objects you will interact using pymongo is Collection, which represents the collection construct in MongoDB. Collections hold groups of related documents.   

You can access collection objects directly using dictionary-style or attribute access from a Database object, as in the following examples:  


```python
db.dataset
db['dataset']
```
You may also assign the collection object to a variable for use elsewhere, as in the following examples:  

```python
coll = db.dataset
coll = db['dataset']
```

In [2]:
config = configparser.ConfigParser()
config.read('./data/mongo.ini')

['./data/mongo.ini']

In [3]:
port = config.get('Mongo', 'port')
print (port)

27017


In [4]:
client = MongoClient(config.get('Mongo', 'host'), int(port))

In [5]:
client.analytics

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'analytics')

In [6]:
client.analytics.collection_names(include_system_collections=False)

['tweets', 'blog', 'users']

In [7]:
try:
    result = client.analytics.blog.drop()
    print ("analytics blog dropped")
except:
    pass

analytics blog dropped


In [8]:
try:
    result = client.analytics.users.drop()
    print ("analytics users dropped")
except:
    pass

analytics users dropped


In [9]:
try:
    result = client.analytics.tweets.drop()
    print ("analytics tweets dropped")
except:
    pass

analytics tweets dropped


In [10]:
client.analytics

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'analytics')

In [11]:
client.analytics.collection_names(include_system_collections=False)

[]

In [12]:
result = client.analytics.users.insert_one({
    "username" : "Goku",
    "firstname" : "Goku",
    "surname" : "Brown",
    "dateofbirth" : datetime(1973, 7, 8),
    "email" : "Goku@GokuBrown.com",
    "score" : 5
})
result.inserted_id

ObjectId('5ac19171a313fc03cdb5052f')

In [13]:
result = client.analytics.blog.insert_one({
    "post_id": 1,
    "author": "Goku",
    "text": "My first blog post!",
    "tags": ["mongodb", "python", "pymongo"],
    "date": datetime.utcnow()
})
result.inserted_id    

ObjectId('5ac19171a313fc03cdb50530')

## You need to be careful; running a script twice may duplicate your data

In [14]:
result = client.analytics.blog.insert_one({
    "post_id": 1,
    "author": "Goku",
    "text": "My first blog post!",
    "tags": ["mongodb", "python", "pymongo"],
    "date": datetime.utcnow()
})
result.inserted_id 

ObjectId('5ac19171a313fc03cdb50531')

In [15]:
db = client.analytics

In [16]:
db.collection_names(include_system_collections=False)

['blog', 'users']

In [17]:
for post in db.blog.find({"author": "Goku"}):
    pprint.pprint(post)
# You need to be careful; running a script twice may duplicate your data    

NameError: name 'pprint' is not defined

### Getting a Single Document With find_one()   

In [ ]:
pprint.pprint(db.blog.find_one())

In [ ]:
pprint.pprint(db.blog.find_one({"author": "Goku"}))

###  Querying By ObjectId   

In [ ]:
mid=db.blog.find_one({"author": "Goku"})['_id'] 
mid

In [ ]:
pprint.pprint(db.blog.find_one({"_id": mid}))

### Bulk Inserts   

In [ ]:
new_posts = [{"author": "Goku",
    "text": "Another post on bulk inserts!",
    "tags": ["bulk", "insert"],
    "date": datetime(2017, 3, 12, 11, 14)},
    {"author": "Vegeta",
    "title": "MongoDB is fun",
    "text": "and pretty easy too!",
    "date": datetime.utcnow()}]
result = db.blog.insert_many(new_posts)
result.inserted_ids

### Querying for More Than One Document  

In [ ]:
for post in db.blog.find():
    pprint.pprint(post)

In [ ]:
for post in db.blog.find({"author": "Goku"}):
    pprint.pprint(post)

### Counting  

In [ ]:
db.blog.count()

In [ ]:
db.blog.find({"author": "Goku"}).count()

### Range Queries

In [ ]:
d = datetime(2017, 5, 5, 12)
for post in db.blog.find({"date": {"$gt": d}}).sort("author"):
    pprint.pprint(post)

### Indexing  

Note if there are duplicates you'll get this error.

```python
DuplicateKeyError: E11000 duplicate key error collection: analytics.users index: username_1 dup key: { : "Goku" }
```

In [ ]:
result = db.users.insert_one({
    "username" : "Vegeta",
    "firstname" : "Vegeta",
    "surname" : "Saiyan",
    "dateofbirth" : datetime(1903, 7, 7),
    "email" : "Vegeta@Saiyan.com",
    "score" : 3
})
result.inserted_id

In [ ]:
for user in db.users.find():
    pprint.pprint(user)

In [ ]:
result = db.users.insert_many([{
    "username" : "Bulma",
    "firstname" : "Bulma",
    "surname" : "Saiyan",
    "dateofbirth" : datetime(1919, 7, 7),
    "email" : "Bulma@Saiyan.com",
    "score" : 2},
{
    "username" : "Trunks",
    "firstname" : "Trunks",
    "surname" : "Saiyan",
    "dateofbirth" : datetime(1955, 7, 7),
    "email" : "Trunks@Saiyan.com",
    "score" : 5
}])

In [ ]:
for user in db.users.find():
    pprint.pprint(user)

In [ ]:
result = db.users.create_index([('username', pymongo.ASCENDING)], unique=True)
sorted(list(db.users.index_information()))

In [ ]:
user_profiles = [
{
    "username" : "Vegeta",
    "firstname" : "Vegeta",
    "surname" : "Saiyan",
    "dateofbirth" : datetime(1903, 7, 7),
    "email" : "Vegeta@Saiyan.com",
    "score" : 3
}]

In [ ]:
pprint.pprint(user_profiles)

In [ ]:
try:
  result = db.users.insert_many(user_profiles)
except:
  print ("insert_many(user_profiles) failed")

_The index prevents us from inserting a document whose username is already in the collection_

### Update  

In [ ]:
import copy
orig_user = db.users.find_one({"username":"Goku"})
new_user = copy.deepcopy(orig_user)
pprint.pprint(new_user)

In [ ]:
# modify the copy to add an email address
l=[]
l.append(new_user["email"])
l.append("Goku@NikBearBrown.com")
l

In [ ]:
new_user["email"] = l
pprint.pprint(new_user)

In [ ]:
# replace the matched document with the contents of new_user
db.users.replace_one({"username":"Goku"}, new_user)

In [ ]:
goku = db.users.find_one({"username":"Goku"})
pprint.pprint(goku)

### Delete  

In [ ]:
for user in db.users.find():
    pprint.pprint(user)

In [ ]:
db.users.delete_one({"score":0})

In [ ]:
for user in db.users.find():
    pprint.pprint(user)

### Query Operators

Queries use a document-style (or python dict) syntax.

In [ ]:
for user in db.users.find({'score': 5}):
    pprint.pprint(user)

In [ ]:
db.users.find({'score': 5}).count()

In [ ]:
for user in db.users.find({'score': {'$lt': 5}}):
    pprint.pprint(user)

In [ ]:
db.users.find({'score': {'$lt': 5}}).count()

In [ ]:
db.users.find_one({'score': {'$lte': 4}})

In [ ]:
import re
rgx = re.compile('Saiyan.com$')

In [ ]:
for user in db.users.find({'email': rgx}):
    pprint.pprint(user)

### Sort results

You can sort the query results on the server side using the sort
method.

In [ ]:
for user in db.users.find({'score': {'$lte': 4}}).sort([('score', pymongo.DESCENDING)]):
    pprint.pprint(user)

In [ ]:
for user in db.users.find({'score': {'$lte': 5}}).sort([('score', pymongo.DESCENDING)]):
    pprint.pprint(user)

### Limit result size  

In [ ]:
for user in db.users.find({'score': {'$lte': 5}}).sort([('score', pymongo.DESCENDING)]).limit(2):
    pprint.pprint(user)

In [ ]:
def load_json(j):
    p=os.path.join("data/", j)
    print (p)
    with open(p, 'rU') as f:
      data = [json.loads(row) for row in f]
    return data 

In [ ]:
tweets_j=load_json('db_tweets.json')
tweets_j[0]

In [ ]:
print(tweets_j[0]['text'])

In [ ]:
tweets = pd.DataFrame(tweets_j)
tweets.head()

In [ ]:
db.tweets

In [ ]:
cnt=0
for tweet in tweets_j:
    print("tweet ", cnt)
    cnt+=1
    pprint.pprint(tweet) 
    print()    

In [ ]:
cnt=0
for tweet in tweets_j:
    print (tweet['id'])    
    print (tweet['text'])    
    print (tweet['created_at'])

In [ ]:
def tweet_json(tid,text,created):
    j={
    "tweet_id" : tid,
    "text" : text,
    "created_at" : created}
    return j

In [ ]:
ts=datetime.now()
# "Current date and time using strftime:"
ts.strftime("%Y-%m-%d %H:%M")

In [ ]:
ts = datetime.strptime('Jun 1 2005  1:33PM', '%b %d %Y %I:%M%p')
ts.strftime("%Y-%m-%d %H:%M")

In [ ]:
# Twitter dates are of the form Sun Mar 15 21:41:54 +0000 2015
datestrings=['Sun Mar 15 21:41:54 +0000 2015','Tue Mar 29 08:11:25 +0000 2011']
from datetime import timedelta
from email.utils import parsedate_tz

def to_datetime(datestring):
    time_tuple = parsedate_tz(datestring.strip())
    dt = datetime(*time_tuple[:6])
    return dt - timedelta(seconds=time_tuple[-1])

ts=to_datetime(datestrings[0])
print (ts.strftime("%Y-%m-%d %H:%M"))
ts=to_datetime(datestrings[1])
print (ts.strftime("%Y-%m-%d %H:%M"))

In [ ]:
cnt=0
for tweet in tweets_j:
    ts=datetime.now()
    try:
        ts=to_datetime(tweet['created_at'])
    except:
        pass     
    try:
        j=tweet_json(tweet['id'],tweet['text'],ts)
        result = db.tweets.insert_one(j)
        cnt+=1
    except:
        pass       
print ("%d tweets inserted."%cnt)

In [ ]:
for tweet in db.tweets.find():
    pprint.pprint(tweet)

Updated October 3, 2017